In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from copy import copy, deepcopy
from datetime import datetime
from tqdm import tqdm
import json

import torch
import torch.nn as nn
import torch.optim as optim

import dataloader as dataloader # dataloader.py
import models as models # models.py
import utils as utils # utils.py

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

import warnings 
warnings.filterwarnings("ignore")

In [2]:
# check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('INFO:')
print("PyTorch Version: ", torch.__version__)
print('GPU State:', device)

INFO:
PyTorch Version:  1.7.0
GPU State: cuda


In [3]:
# hyper parameters
window_size = 6
test_ratio = 0.2
n_batch = 16
n_workers = 4
n_epochs = 300
learning_rate = 0.001
weight_decay = 1e-6
early_stopping_patience = 4

loss_type = 'mae'

grid_size = (10,20) # (H,W)
spatial_hidden_size = 64
temporal_hidden_size = 512

In [4]:
# data loader
def create_dataloader(data, max_value):
    train_dataset = dataloader.NYCTaxiLoader(data=data, mode="train", test_ratio=test_ratio, window_size=window_size, max_value=max_value) 
    train_loader = torch.utils.data.DataLoader(
        dataset=train_dataset, 
        batch_size=n_batch, 
        shuffle=True, 
        num_workers=n_workers
    )
    val_dataset = dataloader.NYCTaxiLoader(data=data, mode="val", test_ratio=test_ratio, window_size=window_size, max_value=max_value) 
    val_loader = torch.utils.data.DataLoader(
        dataset=val_dataset, 
        batch_size=n_batch, 
        shuffle=False, 
        num_workers=n_workers
    )
    test_dataset = dataloader.NYCTaxiLoader(data=data, mode="test", test_ratio=test_ratio, window_size=window_size, max_value=max_value) 
    test_loader = torch.utils.data.DataLoader(
        dataset=test_dataset, 
        batch_size=n_batch, 
        shuffle=False, 
        num_workers=n_workers
    )
    loaders = {'train':train_loader, 'val':val_loader, 'test':test_loader}
    return loaders

In [5]:
# main 
root_path = "./data2"
volume_train = np.load("./data2/volume_train.npz")
volume_test = np.load("./data2/volume_test.npz")
volume_data = np.concatenate([volume_train['volume'], volume_test['volume']], axis=0)
print(f"Total data: {volume_data.shape}\n")

max_value = volume_data[:-int(len(volume_data)*test_ratio)*2].max() # data range [0, max_value] to back transform the predict value
dataloaders = create_dataloader(volume_data, max_value)

Total data: (2880, 10, 20, 2)

> Found 1722 samples in train data...

> Found 570 samples in val data...

> Found 570 samples in test data...



In [6]:
# train
best_loss = 99999
last_loss = -1
times = 0
early_stop = False
loss_dict = {'mse': nn.MSELoss(), 'mae': nn.L1Loss()}

model = models.DMVSTNet(spatial_hidden_size=spatial_hidden_size, temporal_hidden_size=temporal_hidden_size)
model.to(device)
criterion = loss_dict[loss_type]
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=weight_decay)

for epoch in range(n_epochs): 
    if early_stop: 
        print("Early Stop\n")
        break
        
    print(f'Epoch {epoch+1}/{n_epochs}')
    mape_list, rmse_list, train_loss_list, val_loss_list = [], [], [], []
    
    for phase in ['train', 'val']:   
        if phase == 'train':
            model.train()  # Set model to training mode
        elif phase == 'val':
            model.eval()   # Set model to evaluate mode

        running_loss = 0.0
        actual = []
        predicted = []
        
        for batch_x, batch_y in dataloaders[phase]:
            inputs, targets = batch_x.to(device).float(), batch_y.to(device).float()
            
            optimizer.zero_grad()
            spatial_hidden = torch.zeros(window_size, len(inputs), spatial_hidden_size, grid_size[0], grid_size[1]).to(device).float()
            lstm_hidden_cell = (torch.zeros(1, len(inputs), temporal_hidden_size).to(device),
                                torch.zeros(1, len(inputs), temporal_hidden_size).to(device))
            
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs, spatial_hidden, lstm_hidden_cell)
                
                y_true = targets[:,:,:,:,0].view(len(targets),-1)
                y_pred = outputs[0] * max_value # denormalize
                loss = criterion(y_pred, y_true)
               
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                actual.extend(y_true.cpu().detach().numpy())
                predicted.extend(y_pred.cpu().detach().numpy())
                running_loss += loss.item() * inputs.size(0)
        
        # loss
        epoch_loss = running_loss / len(dataloaders[phase].dataset)
        if phase == 'val' and epoch_loss < best_loss:
            best_loss = epoch_loss
            best_model_wts = deepcopy(model.state_dict())
        
        # Early Stopping
        if phase == 'train':
            if epoch_loss > last_loss:
                times += 1
                if times >= early_stopping_patience:
                    early_stop = True
            else:
                times = 0
            last_loss = epoch_loss
            
        # Metrics
        actual, predicted = np.array(actual).T[0], np.array(predicted).T[0]
        rmse = utils.root_mean_squared_error(actual, predicted)
        mape = utils.mean_absolute_percentage_error(actual, predicted)            
        print(f'| {phase} Loss: {epoch_loss:.5f} | rmse: {rmse:.5f} | mape: {mape:.5f}')
        
        mape_list.append(mape)
        rmse_list.append(rmse)
        if phase == 'train':
            train_loss_list.append(epoch_loss)
        else:
            val_loss_list.append(epoch_loss)
    print()


Epoch 1/300
| train Loss: 48.06060 | rmse: 110.00482 | mape: 390.94633
| val Loss: 20.32304 | rmse: 26.79586 | mape: 210.59065

Epoch 2/300
| train Loss: 18.82508 | rmse: 25.97778 | mape: 235.77005
| val Loss: 13.19284 | rmse: 16.29635 | mape: 78.51939

Epoch 3/300
| train Loss: 11.69621 | rmse: 15.17916 | mape: 74.29434
| val Loss: 11.51366 | rmse: 16.05448 | mape: 35.63981

Epoch 4/300
| train Loss: 11.10270 | rmse: 14.86493 | mape: 45.00802
| val Loss: 11.54537 | rmse: 15.41046 | mape: 30.46819

Epoch 5/300
| train Loss: 9.98704 | rmse: 14.97820 | mape: 41.31224
| val Loss: 10.23817 | rmse: 12.46276 | mape: 28.96587

Epoch 6/300
| train Loss: 8.58136 | rmse: 13.90686 | mape: 40.58904
| val Loss: 8.34154 | rmse: 10.73891 | mape: 26.07518

Epoch 7/300
| train Loss: 8.07617 | rmse: 14.02042 | mape: 40.15465
| val Loss: 8.26784 | rmse: 11.85437 | mape: 29.68083

Epoch 8/300
| train Loss: 7.42664 | rmse: 15.13367 | mape: 36.11799
| val Loss: 7.64427 | rmse: 12.54750 | mape: 25.79883

Epo

| train Loss: 5.03981 | rmse: 10.89440 | mape: 25.54054
| val Loss: 5.67586 | rmse: 9.78546 | mape: 22.47125

Epoch 68/300
| train Loss: 5.05548 | rmse: 11.27863 | mape: 25.34459
| val Loss: 6.19460 | rmse: 13.34199 | mape: 24.81590

Epoch 69/300
| train Loss: 5.02765 | rmse: 10.79523 | mape: 25.05183
| val Loss: 6.87410 | rmse: 11.51071 | mape: 22.94833

Epoch 70/300
| train Loss: 5.07624 | rmse: 11.52497 | mape: 26.92565
| val Loss: 6.17139 | rmse: 10.34812 | mape: 23.46977

Epoch 71/300
| train Loss: 4.98783 | rmse: 11.16342 | mape: 26.88572
| val Loss: 5.69141 | rmse: 12.14695 | mape: 23.17959

Epoch 72/300
| train Loss: 5.08543 | rmse: 11.39316 | mape: 26.30534
| val Loss: 5.76196 | rmse: 11.51737 | mape: 22.38334

Epoch 73/300
| train Loss: 5.05409 | rmse: 11.17378 | mape: 26.27241
| val Loss: 5.58147 | rmse: 10.16289 | mape: 22.40748

Epoch 74/300
| train Loss: 5.01716 | rmse: 11.57003 | mape: 26.58361
| val Loss: 5.75974 | rmse: 9.78398 | mape: 22.34703

Epoch 75/300
| train Lo

| train Loss: 4.87878 | rmse: 11.15650 | mape: 26.53351
| val Loss: 5.57968 | rmse: 9.31321 | mape: 21.38675

Epoch 134/300
| train Loss: 4.70417 | rmse: 10.68364 | mape: 25.78432
| val Loss: 5.63382 | rmse: 10.04635 | mape: 20.99702

Epoch 135/300
| train Loss: 4.71022 | rmse: 10.75019 | mape: 24.72049
| val Loss: 5.59327 | rmse: 11.14713 | mape: 21.93817

Epoch 136/300
| train Loss: 4.66113 | rmse: 10.37727 | mape: 24.36219
| val Loss: 5.57246 | rmse: 9.80712 | mape: 21.71749

Epoch 137/300
| train Loss: 4.73097 | rmse: 10.59710 | mape: 25.80669
| val Loss: 5.44589 | rmse: 9.60747 | mape: 21.46405

Epoch 138/300
| train Loss: 4.69901 | rmse: 10.73244 | mape: 25.81696
| val Loss: 5.84538 | rmse: 10.01370 | mape: 21.86599

Epoch 139/300
| train Loss: 4.71237 | rmse: 10.38844 | mape: 24.90382
| val Loss: 5.46693 | rmse: 9.77420 | mape: 21.66922

Epoch 140/300
| train Loss: 4.65358 | rmse: 10.23313 | mape: 24.30060
| val Loss: 5.76508 | rmse: 9.82613 | mape: 20.78778

Epoch 141/300
| tra

| train Loss: 4.39370 | rmse: 9.87823 | mape: 23.29666
| val Loss: 5.51423 | rmse: 12.04402 | mape: 25.32087

Epoch 200/300
| train Loss: 4.37130 | rmse: 9.91336 | mape: 23.89170
| val Loss: 5.27892 | rmse: 9.84392 | mape: 21.38390

Epoch 201/300
| train Loss: 4.49227 | rmse: 10.06502 | mape: 24.39374
| val Loss: 5.60385 | rmse: 11.15955 | mape: 21.59767

Epoch 202/300
| train Loss: 4.37072 | rmse: 10.37967 | mape: 23.75192
| val Loss: 5.41940 | rmse: 10.16996 | mape: 22.23630

Epoch 203/300
| train Loss: 4.43357 | rmse: 9.87599 | mape: 23.66931
| val Loss: 5.46745 | rmse: 11.93773 | mape: 24.70136

Epoch 204/300
| train Loss: 4.49352 | rmse: 9.86846 | mape: 23.62315
| val Loss: 5.53839 | rmse: 10.31961 | mape: 21.49462

Epoch 205/300
| train Loss: 4.50779 | rmse: 10.12170 | mape: 24.60624
| val Loss: 5.48283 | rmse: 12.86638 | mape: 24.49232

Epoch 206/300
| train Loss: 4.41083 | rmse: 10.02602 | mape: 23.68136
| val Loss: 5.44538 | rmse: 10.91543 | mape: 21.88351

Epoch 207/300
| tra

| train Loss: 4.22157 | rmse: 9.29062 | mape: 23.07679
| val Loss: 5.21655 | rmse: 10.56616 | mape: 21.64255

Epoch 266/300
| train Loss: 4.21150 | rmse: 9.64701 | mape: 22.77646
| val Loss: 5.47958 | rmse: 10.90708 | mape: 21.41784

Epoch 267/300
| train Loss: 4.17351 | rmse: 9.31746 | mape: 22.79328
| val Loss: 5.35325 | rmse: 9.80992 | mape: 20.60085

Epoch 268/300
| train Loss: 4.17717 | rmse: 10.02165 | mape: 24.19450
| val Loss: 5.46198 | rmse: 10.66261 | mape: 22.29886

Epoch 269/300
| train Loss: 4.14880 | rmse: 9.38778 | mape: 23.17371
| val Loss: 5.31367 | rmse: 11.39410 | mape: 22.39636

Epoch 270/300
| train Loss: 4.15021 | rmse: 9.95851 | mape: 23.88433
| val Loss: 5.44251 | rmse: 13.14388 | mape: 24.68101

Epoch 271/300
| train Loss: 4.22981 | rmse: 9.40651 | mape: 22.41351
| val Loss: 5.73380 | rmse: 10.58809 | mape: 22.24758

Epoch 272/300
| train Loss: 4.13567 | rmse: 9.18508 | mape: 22.56703
| val Loss: 5.37936 | rmse: 10.40055 | mape: 21.58472

Epoch 273/300
| train 

In [7]:
# output results
from datetime import datetime
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
store_path = f"./results/result_{timestamp}"

# build directory
try:
    os.mkdir(store_path)
except OSError:
    print(f"Creation of the directory {store_path} failed")

# store files
with open(os.path.join(store_path, 'metrics_results.npy'), 'wb') as f:
    np.save(f, np.array(rmse_list))
    np.save(f, np.array(mape_list))
with open(os.path.join(store_path, 'loss_results.npy'), 'wb') as f:
    np.save(f, np.array(train_loss_list))
    np.save(f, np.array(val_loss_list))
torch.save(best_model_wts, os.path.join(store_path, "weight"))

note_json = {
    'window_size': window_size,
    'test_ratio': test_ratio,
    'n_batch': n_batch,
    'n_workers': n_workers,
    'n_epochs': n_epochs,
    'learning_rate': learning_rate,
    'weight_decay': weight_decay,
    'early_stopping_patience': early_stopping_patience,

    'loss': loss_type,
    
    'grid_size': grid_size,
    'spatial_hidden_size': spatial_hidden_size,
    'temporal_hidden_size': temporal_hidden_size
}
with open(os.path.join(store_path, 'note.txt'), 'w') as outfile:
    json.dump(note_json, outfile)

print(f"results stored: {store_path}")

results stored: ./results/result_20201217_130415
